In [1]:
import openai
from qdrant_client import QdrantClient

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

/mnt/c/Users/adity/Documents/workspace-swirlai/ai-engineering-bootcamp-prerequisite/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from dotenv import load_dotenv
import os

load_dotenv("../../.env")

True

### Download an example reference data point from LangSmith

In [9]:
import os
# client = Client()
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [10]:
os.getenv("LANGSMITH_PROJECT")

'rag-tracing'

In [11]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [11]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('266fbf40-a388-447b-aa2b-a106b6c491f8'), created_at=datetime.datetime(2026, 1, 18, 17, 55, 7, 401727, tzinfo=TzInfo(0)), modified_at=datetime.datetime(2026, 1, 18, 17, 55, 7, 401727, tzinfo=TzInfo(0)), example_count=42, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'Windows-11-10.0.26200-SP0', 'sdk_version': '0.6.4', 'runtime_version': '3.12.12', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': None}})

In [12]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': "The UseBean 240W USB-C to USB-C cable (B0BP9Z159S) supports PD3.1 up to 240W and 4K video; it's appropriate for PD fast charging an iPad Pro if your charger and device support PD. The GREPHONE USB-C to Lightning (B0BV6PWVCG) supports PD fast charging for iPhones and some iPads that accept USB‑C to Lightning PD.",
 'reference_context_ids': ['B0BP9Z159S', 'B0BV6PWVCG'],
 'reference_descriptions': ["UseBean 240W USB C to USB C Cable 6.6ft (2 Pack),USB 3.2 Gen 2X2 20Gbps Data Transfer Type-C Cable PD Fast Charging,4K Video Monitor Cord,for Thunderbolt 3/4 MacBook Pro, iPad Pro,Galaxy S21/S20 【240W USB C Cable】Supports PD3.1 fast charging protocol;Supports fast charge up to 240W(48V 5A) when paired up with a compatible cable and charger. Backward compatible with 200/170/140/100/60/18W power devices.(Note: Charging speed may be dependent on the host device's charging capability). 【20Gbps Data Transfer】Offers 20Gbps SuperSpeed data transfer ,transferring an HD movie in just 

In [13]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Can you recommend a cable option here for PD fast charging an iPad Pro?'}

In [14]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

### RAG Pipeline

In [15]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content


def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [16]:
rag_pipeline("Can I get some charger?", top_k=5)

{'answer': 'Yes. Here are charger options currently available:\n- B0BYYLJRHT: 3-pack original Apple MFi certified iPhone Lightning cables, 3 ft each.\n- B0BFPZGYLD: 5-in-1 USB C to Multi Charger Cable, includes Lightning/Type C/Micro USB connectors (10 ft). Note: charging only, not data for some ports.\n- B09TNXY54Y: MUXA 6-pack colorful MFi certified Lightning cables, 3/3/6/6/10/10 ft options.\n- B0BV6PWVCG: GREPHONE 2-pack USB-C to Lightning cables, 6 ft each (MFi certified, supports fast charging with appropriate charger).\n- B0BGDQLZD2: Mixblu replacement charger cables for Fitbit Inspire 3 (2-pack, 3.3 ft).\n\nIf you tell me what device you’re charging and preferred length, I can recommend the best match.',
 'question': 'Can I get some charger?',
 'retrieved_context_ids': ['B0BYYLJRHT',
  'B0BFPZGYLD',
  'B09TNXY54Y',
  'B0BV6PWVCG',
  'B0BGDQLZD2'],
 'retrieved_context': ['iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] Apple MFi Certified USB A Charging 

### RAGAS metrics

In [17]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

C:\Users\adity\AppData\Local\Temp\ipykernel_43684\1973645927.py:2: DeprecationWarning: Importing IDBasedContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextPrecision
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
C:\Users\adity\AppData\Local\Temp\ipykernel_43684\1973645927.py:2: DeprecationWarning: Importing IDBasedContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextRecall
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
C:\Users\adity\AppData\Local\Temp\ipykernel_43684\1973645927.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use

In [18]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

C:\Users\adity\AppData\Local\Temp\ipykernel_43684\2270190839.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
C:\Users\adity\AppData\Local\Temp\ipykernel_43684\2270190839.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [19]:
reference_input

{'question': 'Can you recommend a cable option here for PD fast charging an iPad Pro?'}

In [20]:
reference_output

{'ground_truth': "The UseBean 240W USB-C to USB-C cable (B0BP9Z159S) supports PD3.1 up to 240W and 4K video; it's appropriate for PD fast charging an iPad Pro if your charger and device support PD. The GREPHONE USB-C to Lightning (B0BV6PWVCG) supports PD fast charging for iPhones and some iPads that accept USB‑C to Lightning PD.",
 'reference_context_ids': ['B0BP9Z159S', 'B0BV6PWVCG'],
 'reference_descriptions': ["UseBean 240W USB C to USB C Cable 6.6ft (2 Pack),USB 3.2 Gen 2X2 20Gbps Data Transfer Type-C Cable PD Fast Charging,4K Video Monitor Cord,for Thunderbolt 3/4 MacBook Pro, iPad Pro,Galaxy S21/S20 【240W USB C Cable】Supports PD3.1 fast charging protocol;Supports fast charge up to 240W(48V 5A) when paired up with a compatible cable and charger. Backward compatible with 200/170/140/100/60/18W power devices.(Note: Charging speed may be dependent on the host device's charging capability). 【20Gbps Data Transfer】Offers 20Gbps SuperSpeed data transfer ,transferring an HD movie in just 

In [21]:
result = rag_pipeline(reference_input["question"])

In [22]:
result

{'answer': 'Yes. The UseBean 240W USB C to USB C Cable (ID B0BP9Z159S) supports PD fast charging and is compatible with USB-C devices, including iPad Pro when paired with a compatible charger. It also offers up to 240W PD fast charging and 20Gbps data transfer with 4K video output. If you specifically need a USB-C to USB-C cable for PD charging an iPad Pro, this option fits the requirement.',
 'question': 'Can you recommend a cable option here for PD fast charging an iPad Pro?',
 'retrieved_context_ids': ['B0BP9Z159S',
  'B0BV6PWVCG',
  'B0BFPZGYLD',
  'B0BYYLJRHT',
  'B0BF18F6R7'],
 'retrieved_context': ["UseBean 240W USB C to USB C Cable 6.6ft (2 Pack),USB 3.2 Gen 2X2 20Gbps Data Transfer Type-C Cable PD Fast Charging,4K Video Monitor Cord,for Thunderbolt 3/4 MacBook Pro, iPad Pro,Galaxy S21/S20 【240W USB C Cable】Supports PD3.1 fast charging protocol;Supports fast charge up to 240W(48V 5A) when paired up with a compatible cable and charger. Backward compatible with 200/170/140/100/60

In [23]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [24]:
await ragas_faithfulness(result, "")

1.0

In [26]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [27]:
await ragas_responce_relevancy(result, "")

np.float64(0.7074390642661897)

In [28]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [29]:
await ragas_context_precision_id_based(result, reference_output)

0.4

In [30]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [31]:
await ragas_context_recall_id_based(result, reference_output)

1.0